# Pre requisites

Install and import required packages

In [1]:
!pip install boto3
!pip install tqdm

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/b8/9a/25ed2d1e60835226a87a7437afe42c60ec8ec0b4fa83e04c7b06b1eb3cfb/boto3-1.33.6-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.34.0,>=1.33.6 from https://files.pythonhosted.org/packages/f5/80/20f36e165fc96f33bd4aee4518c32647f308a1b3c3fe9e45afcce5641abe/botocore-1.33.6-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.9.0,>=0.8.2 from https://files.pythonhosted.org/packages/75/ca/5399536cbd5889ca4532d4b8bbcd17efa0fe0be0da04e143667a4ff5644e/s3transfer-0.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.0 MB/s eta 0:00:0000:01
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d088

# New section

In [8]:
import boto3
from pathlib import Path
from botocore import UNSIGNED
from botocore.client import Config
from tqdm.notebook import tqdm

#Data prep things!
!pip install opencv-python
import cv2
import os
import pandas as pd

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a1/f6/57de91ea40c670527cd47a6548bf2cbedc68cec57c041793b256356abad7/opencv_python-4.8.1.78-cp37-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 1.3 MB/s eta 0:00:0000:0100:010m


# Download files from s3 bucket

Get a list of all files in the bucket and download into respective folders within the colab runtime

In [2]:
def get_file_folders(s3_client, bucket_name, prefix=""):
    file_names = []
    folders = []

    default_kwargs = {
        "Bucket": bucket_name,
        "Prefix": prefix
    }
    next_token = ""

    while next_token is not None:
        updated_kwargs = default_kwargs.copy()
        if next_token != "":
            updated_kwargs["ContinuationToken"] = next_token

        response = s3_client.list_objects_v2(**updated_kwargs)
        contents = response.get("Contents")

        for result in contents:
            key = result.get("Key")
            if key[-1] == "/":
                folders.append(key)
            else:
                file_names.append(key)

        next_token = response.get("NextContinuationToken")

    return file_names, folders

In [3]:
def download_files(s3_client, bucket_name, local_path, file_names, folders):
    local_path = Path(local_path)

    for folder in tqdm(folders):
        folder_path = Path.joinpath(local_path, folder)
				# Create all folders in the path
        folder_path.mkdir(parents=True, exist_ok=True)

    for file_name in tqdm(file_names):
        file_path = Path.joinpath(local_path, file_name)
				# Create folder for parent directory
        file_path.parent.mkdir(parents=True, exist_ok=True)
        s3_client.download_file(
            bucket_name,
            file_name,
            str(file_path)
        )

In [4]:
client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
file_names, folders = get_file_folders(client, 'cgiar-crop-damage-classification-challenge')

NameError: name 'boto3' is not defined

In [6]:
download_files(
        client,
        'cgiar-crop-damage-classification-challenge',
        "/content",
        file_names,
        folders
    )

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

#Data pretesting and investigations

In this section, we expand the number of files linked and create datasets for testing etc.

In [ ]:
#Unzip files and do some data preping

!ls
!unzip images.zip



Streaming output truncated to the last 5000 lines.
  inflating: images/4812ceb5bf40a667b840fde0cdd6d2ee697f5122.jpg  
  inflating: images/78f9c227327e8b449acfeedceef06836b061a80f.jpg  
  inflating: images/091db5b0eb6726928d1838a6e5c52e582774ca77.jpg  
  inflating: images/81c74ab6d9c9a12e403d2b5ce2203bcfefa84582.jpg  
  inflating: images/abc90b92fdceec4b58d64b0ff4e35dcb804f47ae.jpg  
  inflating: images/b9b85aa6a00ee96e9d73bd2fa1ce08c1836a42e2.jpg  
  inflating: images/ce6b6a099cbb7d5972f7d61aa43dc582c4cd6226.jpg  
  inflating: images/38a42ae764579d56b0c1a5a3a941250479975606.jpg  
  inflating: images/a0baa422aa4e39969e3db0133ff3269eabfb7ee4.jpg  
  inflating: images/facc477af5ea7f1f8f9364c834f36d4ea2ae38df.jpg  
  inflating: images/968de87f363e87c88b3cee4efa9ab4812ad58b69.jpg  
  inflating: images/299cfe14ff59580cb35bd1b3cfaef8a2c18f8683.jpg  
  inflating: images/4700827a45de2276481df7c72b09f3d65c2aef56.jpg  
  inflating: images/5f7604ca2a96560a4630b31ee5af08d1e761b0da.jpg  
  inflating

In [9]:
#data preping, create a dataframe of the files in folder.



# Define the path to your image dataset
dataset_path = "images/"

# List all image files in the dataset folder
image_files = [f for f in os.listdir(dataset_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Initialize lists to store metadata
file_names = []
dimensions = []
channels = []
image_types = []

# Loop through each image file and gather metadata
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(dataset_path, image_file)

    # Check if the image file exists
    if os.path.exists(image_path):
        # Read the image using OpenCV
        img = cv2.imread(image_path)

        # Get image dimensions (height, width, and number of channels)
        height, width, channel = img.shape

        # Get image type (color or grayscale)
        if channel == 3:
            img_type = "Color"
        elif channel == 1:
            img_type = "Grayscale"
        else:
            img_type = "Unknown"

        # Append metadata to lists
        file_names.append(image_file)
        dimensions.append((height, width))
        channels.append(channel)
        image_types.append(img_type)

# Create a DataFrame from the metadata lists
df = pd.DataFrame({
    'FileName': file_names,
    'Dimensions': dimensions,
    'Channels': channels,
    'ImageType': image_types
})






##Notes:
We need to understand the image dump.

In [10]:
df.head(100)

,FileName,Dimensions,Channels,ImageType
0,000284cf706455209efa6998d380db1cce507b8f.jpg,"(1872, 3952)",3,Color
1,00044f0baa343097fdd89caffdc7559c917c8858.jpg,"(2448, 3264)",3,Color
2,00028878a576cca473b5709e163866fd7d88d3d6.jpg,"(480, 640)",3,Color
3,00008febb86628e3cfdcb87f06f75bc7137802d4.jpg,"(480, 640)",3,Color
4,00044486686a6c02717be696c615b1c04b0cceb4.jpg,"(2448, 3264)",3,Color
5,00099ef52dac1a324f60b1453e20a4ba3c703f8e.jpg,"(480, 640)",3,Color
6,00051b294e6dddeed0cc44dede364ff1537e0443.jpg,"(480, 640)",3,Color
7,000c7cd40f56a8b787808f3fc05fe0db10042a72.jpg,"(240, 320)",3,Color
8,0020a8199fdf7448a845e727640da7cc8e70ad10.jpg,"(480, 640)",3,Color
9,0005d4463459176e42ef54a955504ca933d598b4.jpg,"(480, 640)",3,Color


###Notes:

- will need to resize the frames.
- have to understand img, the variety of image types by doing a [hist](https://docs.opencv.org/3.4/d1/db7/tutorial_py_histogram_begins.html#:~:text=It%20is%20a%20plot%20with,distribution%20etc%20of%20that%20image.)

- Lets implement a simple random forest model.

In [11]:
from PIL import Image
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from tqdm import tqdm

# we have all of the functions!


In [14]:
test_df = pd.read_csv('Test.csv')
train_df = pd.read_csv('Train.csv')

#Merging with our folder dataframe

trainfolder_df = pd.merge(train_df, df, left_on='filename', right_on='FileName', how='inner')

trainfolder_df.head(10)

,ID,damage,filename,FileName,Dimensions,Channels,ImageType
0,ID_DV5KPS,ND,00099ef52dac1a324f60b1453e20a4ba3c703f8e.jpg,00099ef52dac1a324f60b1453e20a4ba3c703f8e.jpg,"(480, 640)",3,Color
1,ID_3IQKNK,G,00008febb86628e3cfdcb87f06f75bc7137802d4.jpg,00008febb86628e3cfdcb87f06f75bc7137802d4.jpg,"(480, 640)",3,Color
2,ID_U85D0M,G,000c7cd40f56a8b787808f3fc05fe0db10042a72.jpg,000c7cd40f56a8b787808f3fc05fe0db10042a72.jpg,"(240, 320)",3,Color
3,ID_VLV64P,WD,0005d4463459176e42ef54a955504ca933d598b4.jpg,0005d4463459176e42ef54a955504ca933d598b4.jpg,"(480, 640)",3,Color
4,ID_FJUGDK,WD,0020a8199fdf7448a845e727640da7cc8e70ad10.jpg,0020a8199fdf7448a845e727640da7cc8e70ad10.jpg,"(480, 640)",3,Color
5,ID_ED9PPD,G,001e9c4f1369c92ded436735f3dc285ce7e3f9ba.jpg,001e9c4f1369c92ded436735f3dc285ce7e3f9ba.jpg,"(480, 640)",3,Color
6,ID_J30XH2,DR,00028878a576cca473b5709e163866fd7d88d3d6.jpg,00028878a576cca473b5709e163866fd7d88d3d6.jpg,"(480, 640)",3,Color
7,ID_CXDEUE,WD,00044f0baa343097fdd89caffdc7559c917c8858.jpg,00044f0baa343097fdd89caffdc7559c917c8858.jpg,"(2448, 3264)",3,Color
8,ID_D2HINW,DR,000284cf706455209efa6998d380db1cce507b8f.jpg,000284cf706455209efa6998d380db1cce507b8f.jpg,"(1872, 3952)",3,Color
9,ID_CUCM8C,DR,00044486686a6c02717be696c615b1c04b0cceb4.jpg,00044486686a6c02717be696c615b1c04b0cceb4.jpg,"(2448, 3264)",3,Color


In [15]:
# we need to join our table

trainfolder_df['damage'].unique

# Display the result DataFrame


<bound method Series.unique of 0    ND
1     G
2     G
3    WD
4    WD
5     G
6    DR
7    WD
8    DR
9    DR
Name: damage, dtype: object>

## Now we join the virtual files just to test   


In [16]:
df.head()

,FileName,Dimensions,Channels,ImageType
0,000284cf706455209efa6998d380db1cce507b8f.jpg,"(1872, 3952)",3,Color
1,00044f0baa343097fdd89caffdc7559c917c8858.jpg,"(2448, 3264)",3,Color
2,00028878a576cca473b5709e163866fd7d88d3d6.jpg,"(480, 640)",3,Color
3,00008febb86628e3cfdcb87f06f75bc7137802d4.jpg,"(480, 640)",3,Color
4,00044486686a6c02717be696c615b1c04b0cceb4.jpg,"(2448, 3264)",3,Color


In [17]:
# We define a function that will read the image file, resize it to the desired size, and return the pixel values as a NumPy array

def read_image(image_path, resize_dim=None):
    # Read the image using OpenCV
    img = cv2.imread(image_path)

    # If the resize dimension is specified,
    # resize the image to the specified dimensions
    if resize_dim is not None:
        height, width = resize_dim
        img = cv2.resize(img, (height, width))

    # Normalize the pixel values
    img = img.astype("float32") / 255.0

    # Return the resized image
    return img

# test = read_image('images/001e9c4f1369c92ded436735f3dc285ce7e3f9ba.jpg', resize_dim=(128, 128))
# test.shape

In [20]:
# join df table with train_df
df_train = pd.merge(train_df, df, left_on='filename', right_on='FileName', how='inner')
df_train.head(10)

,ID,damage,filename,FileName,Dimensions,Channels,ImageType
0,ID_DV5KPS,ND,00099ef52dac1a324f60b1453e20a4ba3c703f8e.jpg,00099ef52dac1a324f60b1453e20a4ba3c703f8e.jpg,"(480, 640)",3,Color
1,ID_3IQKNK,G,00008febb86628e3cfdcb87f06f75bc7137802d4.jpg,00008febb86628e3cfdcb87f06f75bc7137802d4.jpg,"(480, 640)",3,Color
2,ID_U85D0M,G,000c7cd40f56a8b787808f3fc05fe0db10042a72.jpg,000c7cd40f56a8b787808f3fc05fe0db10042a72.jpg,"(240, 320)",3,Color
3,ID_VLV64P,WD,0005d4463459176e42ef54a955504ca933d598b4.jpg,0005d4463459176e42ef54a955504ca933d598b4.jpg,"(480, 640)",3,Color
4,ID_FJUGDK,WD,0020a8199fdf7448a845e727640da7cc8e70ad10.jpg,0020a8199fdf7448a845e727640da7cc8e70ad10.jpg,"(480, 640)",3,Color
5,ID_ED9PPD,G,001e9c4f1369c92ded436735f3dc285ce7e3f9ba.jpg,001e9c4f1369c92ded436735f3dc285ce7e3f9ba.jpg,"(480, 640)",3,Color
6,ID_J30XH2,DR,00028878a576cca473b5709e163866fd7d88d3d6.jpg,00028878a576cca473b5709e163866fd7d88d3d6.jpg,"(480, 640)",3,Color
7,ID_CXDEUE,WD,00044f0baa343097fdd89caffdc7559c917c8858.jpg,00044f0baa343097fdd89caffdc7559c917c8858.jpg,"(2448, 3264)",3,Color
8,ID_D2HINW,DR,000284cf706455209efa6998d380db1cce507b8f.jpg,000284cf706455209efa6998d380db1cce507b8f.jpg,"(1872, 3952)",3,Color
9,ID_CUCM8C,DR,00044486686a6c02717be696c615b1c04b0cceb4.jpg,00044486686a6c02717be696c615b1c04b0cceb4.jpg,"(2448, 3264)",3,Color


In [22]:
!mkdir train

In [23]:
# Resizing the files  and placing them in a folder called train

for index, row in tqdm(df_train.iterrows(), total=df_train.shape[0]):
    # Read the image
    img = read_image(f"images/{row['filename']}", resize_dim=(128, 128))

    # Save the image
    file_path = f"train/{row['filename']}"
    cv2.imwrite(file_path, img)

100%|██████████| 10/10 [00:00<00:00, 80.07it/s]
